## <font style="color:lightblue">Header</font>

### <font style="color:lightblue">Imports</font>

In [ ]:
#%load_ext autoreload
#%autoreload 2

import math
import numpy as np
import torch
import torch.nn as nn
from torchinfo import summary
import itertools
import random
import os

import sinogap_module_long as sg




### <font style="color:lightblue">Redefine</font>

In [ ]:
sg.plt.rcParams['figure.dpi']=223
sg.plt.rcParams['axes.spines.left'] = False
sg.plt.rcParams['axes.spines.right'] = False
sg.plt.rcParams['axes.spines.top'] = False
sg.plt.rcParams['axes.spines.bottom'] = False




### <font style="color:lightblue">Configs</font>

In [ ]:
sg.set_seed(7)

sg.TCfg = sg.TCfgClass(
     exec = 1
    ,nofEpochs = None
    ,latentDim = 64
    ,batchSize = 2**10
    ,batchSplit = 2**2
    ,labelSmoothFac = 0.1 # For Fake labels (or set to 0.0 for no smoothing).
    ,learningRateD = 1e-4
    ,learningRateG = 1e-4
    ,dataDir = '/dev/shm/sinogapData'
)

sg.DCfg = sg.DCfgClass(4)

os.environ["CTAS_MMAP_PATH"] = "/mnt/ssdData/"


### <font style="color:lightblue">Raw Read</font>

In [ ]:
sg.trainSet = sg.createDataSet(sg.TCfg.dataDir, sg.listOfTrainData, exclusive=False)
sg.testSet = sg.createDataSet(sg.TCfg.dataDir+"/test", sg.listOfTestData, exclusive=True)

### <font style="color:lightblue">Show</font>

In [ ]:
sg.refImages, sg.refNoises, sg.refBoxes = sg.createReferences(sg.testSet, 2)
sg.showMe(sg.testSet)


## <font style="color:lightblue">Models</font>

### Generator 2pix

In [ ]:


class Generator2(sg.GeneratorTemplate):

    def __init__(self):
        super(Generator2, self).__init__(2, batchNorm=True)
        self.amplitude = 4 # found by training to optimize it
        self.encoders =  nn.ModuleList([
            self.encblock(  1/self.baseChannels,
                               1, 3, padding=1, norm=False),
            self.encblock(  1, 1, 3, padding=1),
            self.encblock(  1, 2, 3, stride=(2,1), padding=(1,0)),
            self.encblock(  2, 2, 3, padding=1),
            self.encblock(  2, 4, 3, stride=(2,1), padding=(1,0)),
            self.encblock(  4, 4, 3, padding=1),
            self.encblock(  4, 8, 3, stride=(2,1), padding=(1,0)),
            self.encblock(  8, 8, 3, padding=1),
            self.encblock(  8,16, 3, stride=(2,1), padding=(1,0)),
            ])
        self.fcLink = self.createFClink()
        self.decoders = nn.ModuleList([
            self.decblock(32, 8, 3, stride=(2,1), outputPadding=(1,0), padding=(1,0)),
            self.decblock(16, 8, 3, padding=1),
            self.decblock(16, 4, 3, stride=(2,1), outputPadding=(1,0), padding=(1,0)),
            self.decblock( 8, 4, 3, padding=1),
            self.decblock( 8, 2, 3, stride=(2,1), outputPadding=(1,0), padding=(1,0)),
            self.decblock( 4, 2, 3, padding=1),
            self.decblock( 4, 1, 3, stride=(2,1), outputPadding=(1,0), padding=(1,0)),
            self.decblock( 2, 1, 3, padding=1),
            self.decblock( 2, 1, 3, padding=1, norm=False),
            ])
        self.lastTouch = self.createLastTouch()
        sg.load_model(self, model_path="model_2_gen.pt" )


generator2 = Generator2()
generator2 = generator2.to(sg.TCfg.device)
generator2 = generator2.requires_grad_(False)
generator2 = generator2.eval()
sg.lowResGenerators[2] = generator2



### Generator 4pix

In [ ]:
class Generator4(sg.GeneratorTemplate):

    def __init__(self):
        super(Generator4, self).__init__(4, batchNorm=True)
        self.amplitude = 4.5 # found by training to optimize it (4.2)
        self.baseChannels = 4

        def createEncPair(chIn) :
            return [
                self.encblock( chIn,   chIn, 3, padding=(1,0)),
                self.encblock( chIn, 2*chIn, 3, stride=(2,1), padding=(1,0)),
            ]
        self.encoders =  nn.ModuleList([
            self.encblock(  1/self.baseChannels,
                                1, 3, padding=1, norm=False),
            *createEncPair(1),
            *createEncPair(2),
            *createEncPair(4),
            *createEncPair(8),
            self.encblock(16, 16, 3, padding=1),
            self.encblock(16, 16, 3, stride=(2,1), padding=(1,0)),
            ])

        self.fcLink = self.createFClink()

        def createDecPair(chIn) :
            return [
                self.decblock(4*chIn, chIn, 3, stride=(2,1), outputPadding=(1,0), padding=(1,0)),
                self.decblock(2*chIn, chIn, 3, padding=(1,0)),
            ]
        self.decoders = nn.ModuleList([
            self.decblock(32, 16, 3, stride=(2,1), outputPadding=(1,0), padding=(1,0)),
            self.decblock(32, 16, 3, padding=1),
            *createDecPair(8),
            *createDecPair(4),
            *createDecPair(2),
            *createDecPair(1),
            self.decblock( 2, 1, 3, padding=1, norm=False),
            ])
        self.lastTouch = self.createLastTouch()
        #sg.load_model(self, model_path="model_3_gen.pt" )


generator4 = Generator4()
generator4 = generator4.to(sg.TCfg.device)
#generator2 = generator2.requires_grad_(False)
generator4 = generator4.eval()
sg.lowResGenerators[4] = generator4

#model_summary = summary(generator4, input_data=[ [sg.refImages[[0],...], sg.refNoises[[0],...]] ] ).__str__()
#print(model_summary)




### <font style="color:lightblue">Generator</font>

In [ ]:
sg.generator = sg.lowResGenerators[sg.DCfg.gapW]
sg.optimizer_G = sg.createOptimizer(sg.generator, sg.TCfg.learningRateG)
model_summary = summary(sg.generator, input_data=[ [sg.refImages[[0],...], sg.refNoises[[0],...]] ] ).__str__()
print(sg.refImages.shape, sg.refImages[[0],...].shape)
print(model_summary)


### <font style="color:lightblue">Discriminator</font>

In [ ]:

class Discriminator(sg.DiscriminatorTemplate):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.param = nn.Parameter(torch.zeros(1))
    def forward(self, images):
        return torch.zeros((images.shape[0],1), device=sg.TCfg.device)

sg.discriminator = Discriminator()
sg.discriminator = sg.discriminator.to(sg.TCfg.device)
model_summary = summary(sg.discriminator, input_data=sg.refImages[0,...] ).__str__()
print(model_summary)
#sg.writer.add_graph(sg.discriminator, refImages)

sg.optimizer_D = sg.createOptimizer(sg.discriminator, sg.TCfg.learningRateD)



## <font style="color:lightblue">Norms</font>

In [ ]:
sg.dataLoader = sg.createDataLoader(sg.trainSet, shuffle=True, num_workers=16)
sg.testLoader = sg.createDataLoader(sg.testSet, shuffle=False, num_workers=16)

#_ = sg.summarizeMe(sg.testLoader, True)
#_ = sg.summarizeMe(sg.testLoader, False)
#_ = sg.summarizeMe(sg.dataLoader, True)
#_ = sg.summarizeMe(sg.dataLoader, False)
#sg.displayImages()

### no prenorm

sg.metrices = {
    'Adv'    : sg.Metrics(sg.loss_Adv_Gen, 0.000e+00, 0),
    'MSE'    : sg.Metrics(sg.loss_MSE,     1.570e+00, 1),
    'MSEN'   : sg.Metrics(sg.loss_MSEN,    1.362e+02, 0),
    'L1L'    : sg.Metrics(sg.loss_L1L,     3.180e+01, 0),
    'L1LN'   : sg.Metrics(sg.loss_L1LN,    3.149e+02, 0),
    'SSIM'   : sg.Metrics(sg.loss_SSIM,    2.039e-03, 0),
    'MSSSIM' : sg.Metrics(sg.loss_MSSSIM,  3.308e-05, 0),
}

#sg.metricesTrain = {
#    'Adv'    : sg.Metrics(sg.loss_Adv_Gen, 0.000e+00, 0),
#    'MSE'    : sg.Metrics(sg.loss_MSE,     3.300e+00, 1),
#    'L1L'    : sg.Metrics(sg.loss_L1L,     5.319e+01, 1),
#    'SSIM'   : sg.Metrics(sg.loss_SSIM,    1.929e-03, 1),
#    'MSSSIM' : sg.Metrics(sg.loss_MSSSIM,  4.757e-05, 1),
#}


## <font style="color:lightblue">Restore checkpoint</font>

In [ ]:

sg.scheduler_G = torch.optim.lr_scheduler.StepLR(sg.optimizer_G, 1, gamma=1)
#sg.scheduler_D = torch.optim.lr_scheduler.StepLR(sg.optimizer_D, 1, gamma=1)
savedCheckPoint = f"checkPoint_{sg.TCfg.exec}"
sg.epoch, sg.imer, sg.minGEpoch, sg.minGdLoss, sg.startFrom, sg.resAcc = \
    sg.restoreCheckpoint(savedCheckPoint+".pth")
#sg.epoch, sg.imer, sg.minGEpoch, sg.minGdLoss, sg.startFrom = 0, 0, 0, 1, 0
sg.writer = sg.createWriter(sg.TCfg.logDir, True)
#sg.writer.add_graph(sg.generator, ((sg.refImages, sg.refNoises),) )
#sg.writer.add_graph(sg.discriminator, refImages)



sg.scheduler_G.gamma = 1-0.01
#sg.scheduler_D.gamma = 1-0.005
#sg.optimizer_G.step()
#torch.optim.lr_scheduler.LambdaLR(sg.optimizer_G, lambda epoch: 1).step()
#sg.optimizer_D.step()
#torch.optim.lr_scheduler.LambdaLR(sg.optimizer_D, lambda epoch: 0.05).step()

print(f"Initial LR : G {sg.scheduler_G.get_last_lr()[0]:.3e} ({sg.scheduler_G.get_last_lr()[0]/sg.TCfg.learningRateG:.4f})")
                  #f" D {sg.scheduler_D.get_last_lr()[0]:.3e} ({sg.scheduler_D.get_last_lr()[0]/sg.TCfg.learningRateD:.4f}), Epoch {sg.epoch} {sg.imer}")


In [ ]:
sg.displayImages()
#_ = sg.summarizeMe(sg.testLoader, False)

## <font style="color:lightblue">Execute</font>

In [ ]:


sg.scheduler_G.gamma = 1-0.005
def my_beforeReport(sglocals) :
    message = ""

    message += "G" + sg.adjustScheduler(sg.scheduler_G, sg.TCfg.learningRateG, 0.1)
    sg.writer.add_scalars("Aux", {'LrnRate': sg.scheduler_G.get_last_lr()[0]/sg.TCfg.learningRateG}, sg.imer)

    #message += f"Amplitude {sg.generator.amplitude.item()}."
    #sg.writer.add_scalars("Aux", {'Amplitude': sg.generator.amplitude.item()}, sg.imer)

    #for metrics in ('L1L','SSIM','MSSSIM') :
    #    sg.metrices[metrics].weight = max(0, sg.metrices[metrics].weight - 0.005)
    #    message += f"{metrics} weight: {sg.metrices[metrics].weight}. "
    #    sg.writer.add_scalars("Aux", {metrics: sg.metrices[metrics].weight}, sg.imer)

    #with open(f"message_{sg.TCfg.exec}.txt", 'a') as file:
    #    file.write(message + "\n")
    print(message)
    return
#sg.beforeReport = my_beforeReport


try :
    sg.train(savedCheckPoint)
except :
    #del sg.dataLoader
    #del sg.testLoader
    sg.freeGPUmem()
    1/10 # to release Jupyuter memory in the next step
    sg.epoch -= 1
    raise

#0 norm, BS=2**10, LR=1e-4


## <font style="color:lightblue">Post</font>

In [ ]:
sg.saveCheckPoint(savedCheckPoint+".pth",
                           sg.epoch+1, sg.imer, sg.minGEpoch, sg.minGLoss,
                           sg.generator, sg.discriminator,
                           sg.optimizer_G, sg.optimizer_D)


In [ ]:
resTest = sg.summarizeMe(sg.testLoader, False)
resTest *= 1/resTest.nofIm
sg.writer.add_scalars("Losses test",
                   {'Dis': resTest.lossD
                   ,'Gen': resTest.lossG
                   ,'Adv' : resTest.metrices['Adv']
                   }, sg.epoch+1 )
sg.writer.add_scalars("Metrices test", resTest.metrices, sg.epoch )
sg.writer.add_scalars("Probs test",
                   {'Ref':resTest.predReal
                   ,'Gen':resTest.predFake
                   #,'Pre':trainRes.predGen
                   }, sg.epoch+1 )





In [ ]:
sg.testMe(trainSet, 5)

### <font style="color:lightblue">Save results</font>

In [ ]:
sg.saveModels()